In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from numpy.random import randint
from numpy.random import rand
import numpy as np

pd.options.display.max_columns = 30
%matplotlib inline

### TF-IDF Vector

In [ ]:
list = []

In [ ]:
#Reading our data
infile = open('train-data_new.dat', 'r')
lines = infile.readlines()
infile.close()

for str in lines:
    list.append(str)
    

In [ ]:
new_list = [s.replace("\n", "").strip() for s in list]

In [ ]:
new_list[0]

In [ ]:
count_vectorizer = CountVectorizer()

In [ ]:
# .fit_transform TOKENIZES and COUNTS
X = count_vectorizer.fit_transform(new_list)

In [ ]:
X

In [ ]:
X.toarray()

In [ ]:
pd.DataFrame(X.toarray())

In [ ]:
count_vectorizer.get_feature_names()

In [ ]:
pd.DataFrame(X.toarray(), columns=count_vectorizer.get_feature_names())

In [ ]:
#tf values
tfidf_vectorizer = TfidfVectorizer(stop_words='english',use_idf=False)
X = tfidf_vectorizer.fit_transform(new_list)
tf = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names())

In [ ]:
#idf values
# use_idf=True is default
idf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True)
X = idf_vectorizer.fit_transform(new_list)
idf_df = pd.DataFrame(X.toarray(), columns=idf_vectorizer.get_feature_names())
idf_df

In [ ]:
#just a check for specific values
tf['674'], idf_df['674']

In [ ]:
# Multiply two DataFrames to make the tfidf values

tfidf = tf.mul(idf_df);
print(tfidf);

In [ ]:
#check
tfidf["674"]

In [ ]:
#mean tfidf value for every single word
tfidf = tfidf.mean()

In [ ]:
tfidf

In [ ]:
#turning it into DataFrame
tfidf_df = pd.DataFrame([tfidf])

In [ ]:
tfidf_df = tfidf_df*100000
tfidf_df

### Initial Population

In [ ]:
n_pop = 20

In [ ]:
n_bits = 8510

In [ ]:
#Creating 4 smaller populations with different probabilities to get various chromosomes
pop1 = [np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop2 = [np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop3 = [np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=n_bits).tolist() for _ in range(int(n_pop/4))]
pop4 = [np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=n_bits).tolist() for _ in range(int(n_pop/4))]

In [ ]:
pop = pop1 + pop2 + pop3 + pop4

In [ ]:
pop

In [ ]:
pop_df = pd.DataFrame(pop, columns=idf_vectorizer.get_feature_names())

In [ ]:
pop_df

In [ ]:
#Changing the value of every selected word from 1 to its tfidf value
tfidf_values = pd.DataFrame(columns=idf_vectorizer.get_feature_names());
    
for i in range(len(pop_df)) :
    tfidf_values = tfidf_values.append(pop_df.iloc[i,:]*tfidf_df)

In [ ]:
tfidf_values

In [ ]:
#Mean tfidf value of all selected words for every chromosome
tfidf_mean = tfidf_values.sum(axis = 1)
tfidf_mean = tfidf_mean/(tfidf_values > 0).sum(axis = 1)
tfidf_mean

In [ ]:
selected_words = (tfidf_values > 0).sum(axis = 1)
selected_words

#### Penalty Function

In [ ]:
def penalty(x):
        return (abs(x-1000)/100)

In [ ]:
for s in selected_words:
    print(format(penalty(s), ".3f"))

In [ ]:
s1 = pd.Series(selected_words, name='selected_words')

s2 = pd.Series(tfidf_mean, name='tfidf')

final = pd.concat([s1, s2], axis=1)
final = final.reset_index()
final = final.drop('index', 1)
final

#### Fitness Function

In [ ]:
def fitness(df):
    x = []
    for i in range(len(df)) :
        if df.iloc[i,0] < 1000:
            x.append([0])
        else:
            x.append([df.iloc[i,1]**2 - penalty(df.iloc[i,0])])
    return pd.DataFrame(x, columns = ['fit'])

In [ ]:
fitness(final)

### Selection Function / Roulette Wheel

In [ ]:
def roulette_wheel_selection(population):

    # Computes the totallity of the population fitness
    population_fitness = fitness(population).sum()
    
    chromosome_probabilities = pd.DataFrame()
    x = fitness(population)
    
    # Computes for each chromosome the probability 
    for row in x.itertuples():
        chromosome_probabilities = chromosome_probabilities.append(row.fit/population_fitness, ignore_index=True)
    
    chromosome_probabilities.rename(columns = {'fit':'Prob'}, inplace = True)
    #np.random.choice(np.arange(0, len(population)), p=[chromosome_probabilities.iloc[:,0].values])
    # Returns the probabilities for every chromosome in the roulette
    return chromosome_probabilities

In [ ]:
#np.random.choice(np.arange(0, len(final)), p=[x.iloc[:,0].values])
#x = roulette_wheel_selection(final)
#np.array(x.iloc[:,0].values)

In [ ]:
roulette_wheel_selection(final).sum()

### Crossover Function / Single Point

In [ ]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

### Mutation Function

In [ ]:
# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


### GA IMPLEMENTATION-NOT FINISHED

In [ ]:
# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop1 = [np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop2 = [np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop3 = [np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop4 = [np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=n_bits).tolist() for _ in range(int(n_pop/4))]
    pop = pop1 + pop2 + pop3 + pop4
    pop_df = pd.DataFrame(pop, columns=idf_vectorizer.get_feature_names())
    
    # keep track of best solution
    best, best_eval = 0, objective(pop_df[0])
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c) for c in pop_df]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop_df[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop_df[i], scores[i]))
        # select parents
        selected = [roulette_wheel_selection(pop_df, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop_df = children
    return [best, best_eval]


In [ ]:
# define the total iterations
n_iter = 100
# bits
n_bits = 8510
# define the population size
n_pop = 20
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)
# perform the genetic algorithm search
best, score = genetic_algorithm(penalty, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))